In [8]:
from pytorch3d.transforms import euler_angles_to_matrix, matrix_to_euler_angles, se3_log_map, se3_exp_map
import torch
import math
from src.util.cameras import PatchPerspectiveCameras as PatchCameras

In [12]:
focal_length = torch.tensor([-1252.8131])
principal_point = torch.tensor([[826.5881, 469.9846]])
image_size = torch.tensor([[900, 1600]])
camera = PatchCameras(focal_length=focal_length,
            principal_point=principal_point,
            znear=0,
            zfar=80,
            device="cuda" if torch.cuda.is_available() else "cpu",
            image_size=image_size)

In [16]:
# add v1, v2, v3 to bbox_means and bbox_logvars
yaw_std = torch.tensor([math.pi])
# pitch and roll are always zero
# XYZ convention: 

euler_mean = torch.tensor([0., 0., 0.], dtype=torch.float32)
euler_std = torch.tensor([0., yaw_std, 0.], dtype=torch.float32)

convention = "XYZ"

R_mean = euler_angles_to_matrix(euler_mean, convention)
R_std = euler_angles_to_matrix(euler_std, convention)

patch_size = torch.tensor([256, 256], dtype=torch.float32)
    
patch_center = torch.tensor([967.3616, 491.6377], dtype=torch.float32)
world_to_patch_transform = camera.get_patch_projection_transform(patch_size=patch_size, patch_center=patch_center)

# R_mean_patch = world_to_patch_transform.transform_normals(normals=R_mean) # TODO: Inverting error
# R_std_patch = world_to_patch_transform.transform_normals(normals=R_std) # TODO: Inverting error
R_patch_mean = R_mean # TODO: R should be in patch NDC
R_patch_std = R_std # TODO: R should be in patch NDC

translation = torch.tensor([0., 0., 0.], dtype=torch.float32)
# A SE(3) matrix has the following form: ` [ R 0 ] [ T 1 ] , `
se3_exp_map_matrix_mean = torch.eye(4)
se3_exp_map_matrix_mean[:3, :3] = R_patch_mean
se3_exp_map_matrix_mean[:3, 3] = translation

se3_exp_map_matrix_std = torch.eye(4)
se3_exp_map_matrix_std[:3, :3] = R_patch_std
se3_exp_map_matrix_std[:3, 3] = translation

# form must be ` [ R 0 ] [ T 1 ] , ` so need to transpose
se3_exp_map_matrix_mean = se3_exp_map_matrix_mean.T
se3_exp_map_matrix_std = se3_exp_map_matrix_std.T

# add batch dim if not present
if se3_exp_map_matrix_mean.dim() == 2:
    se3_exp_map_matrix_mean = se3_exp_map_matrix_mean.unsqueeze(0)
    
if se3_exp_map_matrix_std.dim() == 2:
    se3_exp_map_matrix_std = se3_exp_map_matrix_std.unsqueeze(0)

pose_6d_mean = se3_log_map(se3_exp_map_matrix_mean) # 6d vector
pose_6d_std = se3_log_map(se3_exp_map_matrix_std) # 6d vector

screen_to_ndc_transform in cameras tensor([[[ 0.0022,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0022,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  1.0000,  0.0000],
         [-1.8959, -1.0888,  0.0000,  1.0000]]])
cx_screen tensor([[967.3616]]) torch.Size([1, 1])
point_screen tensor([[967.3616, 491.6377,   1.0000]]) torch.Size([1, 3])
cx_ndc tensor([0.2537]) torch.Size([1])
cy_ndc tensor([0.0037]) torch.Size([1])
principal_point in cameras tensor([[826.5881, 469.9846]])
scale tensor([900.])
patch_scale tensor([256.])
tx tensor([0.2537])
ty tensor([0.0037])


In [18]:
pose_6d_mean, pose_6d_std

(tensor([[0., 0., 0., 0., 0., 0.]]),
 tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -3.8761e-05,
           0.0000e+00]]))